In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls 'drive/My Drive'

'Colab Notebooks'


In [0]:
import os
import numpy as np
import keras

Using TensorFlow backend.


In [0]:
from keras.utils import np_utils
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Concatenate, GlobalAveragePooling2D, AveragePooling2D, Dense
from keras.callbacks import ModelCheckpoint, CSVLogger

In [0]:
main_path = 'drive/My Drive/Colab Notebooks/Kuzushiji/Kuzushiji-49'
n_class = 49
img_shape = (28, 28, 1)

In [0]:
train_X_file = np.load(os.path.join(main_path, 'k49-train-imgs.npz'))
train_Y_file = np.load(os.path.join(main_path, 'k49-train-labels.npz'))

In [0]:
train_X = np.expand_dims(train_X_file['arr_0'], axis = 3) / 255.0
train_Y = np_utils.to_categorical(train_Y_file['arr_0'], n_class)

CNNを構築

In [0]:
def dense_block(x, k, n_block):
  for i in range(n_block):
    main = x
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #1x1
    x = Conv2D(filters = 64, kernel_size = (1, 1), padding = 'valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #3x3
    x = Conv2D(filters = k, kernel_size = (3, 3), padding = 'same')(x)
    #concat
    x = Concatenate()([main, x])
  return x

In [0]:
def transition_layer(inputs, compression = 0.5):
  n_channel = int(inputs.shape[3])
  filters = int(n_channel * compression)
  x = Conv2D(filters = filters, kernel_size = (1, 1))(inputs)
  outputs = AveragePooling2D(pool_size = (2, 2))(x)
  
  return outputs

In [0]:
def DenseNet():
  inputs = Input(shape = (28, 28, 1))

  x = dense_block(inputs, k = 16, n_block = 1)
  x = transition_layer(x, compression = 0.5)
  x = dense_block(x, k = 16, n_block = 2)
  x = transition_layer(x, compression = 0.5)
  x = dense_block(x, k = 16, n_block = 4)
  x = transition_layer(x, compression = 0.5)
  x = dense_block(x, k = 16, n_block = 3)

  x = GlobalAveragePooling2D()(x)

  x = Dense(512)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Dense(n_class)(x)

  outputs = Activation('softmax')(x)

  return Model(inputs, outputs)

In [0]:
model = DenseNet()

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
optimizer = keras.optimizers.Adam()

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 28, 28, 1)    4           input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 28, 28, 1)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 64)   128         activation_1[0][0]               
__________________________________________________________________________________________________
batch_norm

In [0]:
save_path = os.path.join(main_path, 'model')
CNN_name = 'DenseNet_2'

In [0]:
if not os.path.exists(save_path):
  os.mkdir(save_path)

if not os.path.exists(os.path.join(save_path, CNN_name)):
  os.mkdir(os.path.join(save_path, CNN_name))

In [0]:
checkpoint = ModelCheckpoint(os.path.join(save_path, CNN_name, CNN_name + '_{epoch:d}.h5'))
csv_logger = CSVLogger(os.path.join(save_path, CNN_name, CNN_name + '.csv'), append = True)

In [0]:
model.fit(train_X, train_Y, epochs = 100, callbacks = [checkpoint, csv_logger], validation_split = 0.1)